# HW2- Binary Classification 
- Detravious Jamari Brinkley
- CSCI-544: Applied Natural Language Processing
- python version: 3.11.4

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup

from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer

from sklearn.model_selection import train_test_split

import sklearn
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/brinkley97/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read Data

In [2]:
dataset = "../datasets/amazon_reviews_us_Office_Products_v1_00.tsv"
amazon_reviews_copy_df = pd.read_csv(dataset, sep='\t', on_bad_lines='skip', low_memory=False)

## Keep Reviews and Ratings

In [3]:
reviews_ratings_df = amazon_reviews_copy_df.loc[0:, ['star_rating', 'review_body']]
reviews_ratings_df.reset_index(drop=True)

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


In [4]:
def generate_sample_reviews(df: pd.DataFrame, review_col_name: str, number_of_reviews: int = 3):
    """Include reviews and ratings

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    review_col_name: `str`
        The specific_column to get the reviews and ratings of
    
    number_of_reviews: `int`
        Number of samples to include


    Return
    ------
    Nothing; instead, print the reviews with ratings
    """


    columns_to_include = [review_col_name, 'star_rating']

    # Initialize an empty list to store dictionaries
    list_of_dicts = []

    # Iterate over the specified columns and retrieve the first three rows
    for row in df[columns_to_include].head(3).to_dict(orient='records'):
        list_of_dicts.append({'star_rating': row['star_rating'], review_col_name: row[review_col_name]})

    for dictionary in list_of_dicts:
        print(dictionary)

 ## Select 100000 reviews randomly from positive and negative classes


In [5]:
def update_data_type(df: pd.DataFrame, col_name: str):
    """Update the data type of the star ratings

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with rating values

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the new sentiment appened

    """

    valid_ratings = ['1','2','3','4','5']
    star_rating_series = df[col_name].copy()

    # Convert type to strings
    star_rating_series.astype('str')

    # Check valid list and see which of our stars match
    rows = star_rating_series.index
    is_rating_in_valid_ratings = rows[star_rating_series.isin(valid_ratings)]

    # Convert to list
    is_rating_in_valid_ratings = is_rating_in_valid_ratings.to_list()

    updated_df = df.iloc[is_rating_in_valid_ratings]
    updated_df[col_name] = updated_df[col_name].astype(int)
    return updated_df

In [6]:
updated_reviews_ratings_df = update_data_type(reviews_ratings_df, 'star_rating')

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_44955/2907883124.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  updated_df[col_name] = updated_df[col_name].astype(int)


In [7]:
updated_reviews_ratings_df

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


In [8]:
updated_reviews_ratings_df = updated_reviews_ratings_df.dropna()
updated_reviews_ratings_df

,star_rating,review_body
0,5,Great product.
1,5,What's to say about this commodity item except...
2,5,"Haven't used yet, but I am sure I will like it."
3,1,Although this was labeled as &#34;new&#34; the...
4,4,Gorgeous colors and easy to use
...,...,...
2640249,4,I can't live anymore whithout my Palm III. But...
2640250,4,Although the Palm Pilot is thin and compact it...
2640251,4,This book had a lot of great content without b...
2640252,5,I am teaching a course in Excel and am using t...


In [9]:
# Check for NaN values
nan_check = updated_reviews_ratings_df.isna()

# Display the DataFrame with True where NaN values exist
print(nan_check)

# Check if any NaN value exists in the DataFrame
if nan_check.any().any():
    print("There are NaN values in the DataFrame.")
else:
    print("There are no NaN values in the DataFrame.")

         star_rating  review_body
0              False        False
1              False        False
2              False        False
3              False        False
4              False        False
...              ...          ...
2640249        False        False
2640250        False        False
2640251        False        False
2640252        False        False
2640253        False        False

[2640080 rows x 2 columns]
There are no NaN values in the DataFrame.


In [10]:
print("# reviews per rating", updated_reviews_ratings_df['star_rating'].value_counts())

# reviews per rating star_rating
5    1582704
4     418348
1     306967
3     193680
2     138381
Name: count, dtype: int64


In [11]:
def sample_star_ratings(df: pd.DataFrame, col_name: str, star_value: int, number_of_reviews: int):
    """Build a subset balanced dataset with reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The dataframe to use
    col_name: `str`
        The name of the column to get reviews from
    star_value: `int`
        The star rating of the review
    number_of_reviews: `int`
        The number of sub reviews to include in sample

    Return
    ------
    rating_df, sampled_rating_df: `tuple`
        All reviews with that rating and the subset reviews with that rating
    """
    
    rating_df = df[df[col_name] == star_value]
    sampled_rating_df = rating_df.sample(n=number_of_reviews)
    return rating_df, sampled_rating_df

In [12]:
subset_reviews = 50000

one_star = 1
rating_one, rating_one_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', one_star, subset_reviews)
two_stars = 2
rating_two, rating_two_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', two_stars, subset_reviews)
three_stars = 3
rating_three, rating_three_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', three_stars, subset_reviews)
four_stars = 4
rating_four, rating_four_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', four_stars, subset_reviews)
five_stars = 5
rating_five, rating_five_sampled = sample_star_ratings(updated_reviews_ratings_df, 'star_rating', five_stars, subset_reviews)

In [13]:
sampled_reviews_df = pd.concat([rating_one_sampled, rating_two_sampled, rating_three_sampled, rating_four_sampled, rating_five_sampled])

In [14]:
sampled_reviews_df

,star_rating,review_body
2593858,1,The misleading promises of an enormous word an...
368879,1,"Would not work for my HP printer, incompatible..."
1731809,1,1 month later and already getting low toner me...
2632115,1,"I'm a gadget freak, so I always have to have t..."
2571679,1,this printer is junk. <br />after a couple mon...
...,...,...
1354188,5,I prefer to right in green pen when I have the...
2135939,5,As much as I appreciate electronic gradebooks ...
433773,5,Thanks for the excellent ink cartridges. We d...
1630532,5,"This is a great item, it's sturdy and solid, a..."


In [15]:

def separate_reviews_by_rating(df: pd.DataFrame, rating_col: str, threshold: int, sentiment_type: str):
    """Categorizes reviews by adding a rating

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    rating_col: `str`
        Column with rating values
    
    threshold: `int`
        Where to split the ratings such that categories can be formed

    sentiment_type: `str`
        One of three types of sentiment: positive, negative, or neural

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the new sentiment appened
    """


    if sentiment_type == 'negative_review_class':
        positive_review_threshold = df[rating_col].astype('int32') > threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 1

    elif sentiment_type == 'neutral_review_class':
        positive_review_threshold = df[rating_col].astype('int32') == threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 2

    elif sentiment_type == 'positive_review_class':
        positive_review_threshold = df[rating_col].astype('int32') < threshold
        df = df[positive_review_threshold]
        df[sentiment_type] = 3
        
    return df

In [16]:
negative_review_class_df = separate_reviews_by_rating(sampled_reviews_df, 'star_rating', 3, 'negative_review_class')
neutral_review_class_df = separate_reviews_by_rating(sampled_reviews_df, 'star_rating', 3, 'neutral_review_class')
positive_review_class_df = separate_reviews_by_rating(sampled_reviews_df, 'star_rating', 3, 'positive_review_class')

/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_44955/2166075515.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 1
/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_44955/2166075515.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[sentiment_type] = 2
/var/folders/fz/zn5r8vq12nv5p23dtlr15sk40000gn/T/ipykernel_44955/2166075515.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_in

In [17]:
sampled_reviews_ratings_df = pd.concat([negative_review_class_df, neutral_review_class_df, positive_review_class_df])
sampled_reviews_ratings_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class
254374,4,The pens cam on time and I got the right numbe...,1.0,NaN,NaN
1516628,4,Hey... It's a roll of STICKERS. What else can...,1.0,NaN,NaN
885182,4,It is as appears in the picture. It accommodat...,1.0,NaN,NaN
2270302,4,I really loved this laser pointer. It is extr...,1.0,NaN,NaN
1278832,4,In use with my Amazon Basics 3-Button corded U...,1.0,NaN,NaN
...,...,...,...,...,...
1810597,2,"First of all, let me say I am extremely happy ...",NaN,NaN,3.0
2349422,2,This headset is amazing but the mic that you f...,NaN,NaN,3.0
235316,2,Smaller than expected. Didn't end up using the...,NaN,NaN,3.0
603714,2,it's nice that it collapses to about the size ...,NaN,NaN,3.0


In [18]:
negative_reviews_df = sampled_reviews_ratings_df['negative_review_class'].dropna()
neutral_reviews_df = sampled_reviews_ratings_df['neutral_review_class'].dropna()
positive_reviews_df = sampled_reviews_ratings_df['positive_review_class'].dropna()

In [19]:
sampled_reviews_ratings_df['review_class'] = pd.concat([negative_reviews_df, neutral_reviews_df, positive_reviews_df])

In [20]:
sampled_reviews_ratings_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class
254374,4,The pens cam on time and I got the right numbe...,1.0,NaN,NaN,1.0
1516628,4,Hey... It's a roll of STICKERS. What else can...,1.0,NaN,NaN,1.0
885182,4,It is as appears in the picture. It accommodat...,1.0,NaN,NaN,1.0
2270302,4,I really loved this laser pointer. It is extr...,1.0,NaN,NaN,1.0
1278832,4,In use with my Amazon Basics 3-Button corded U...,1.0,NaN,NaN,1.0
...,...,...,...,...,...,...
1810597,2,"First of all, let me say I am extremely happy ...",NaN,NaN,3.0,3.0
2349422,2,This headset is amazing but the mic that you f...,NaN,NaN,3.0,3.0
235316,2,Smaller than expected. Didn't end up using the...,NaN,NaN,3.0,3.0
603714,2,it's nice that it collapses to about the size ...,NaN,NaN,3.0,3.0


# Ignore below

# Data Cleaning

## Lower case

In [21]:
def convert_reviews_to_lower_case(df: pd.DataFrame, col_name: str):
    """Convert all reviews to lower case

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the lower cased reviews
    """
    
    lower_case_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    
    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        # print(text_reviews_idx, type(text_review), text_review)

        # NOT all reviews are strings, thus all can't be converted to lower cased
        if type(text_review) != str:
            print(True, text_review)
            converted_str = str(text_review)
            lower_case_reviews.append(text_review)
         
        else:
            update_text_review = text_review.lower()
            lower_case_reviews.append(update_text_review)

    updated_df['lower_cased'] = lower_case_reviews
    return updated_df

In [22]:
reviews_lower_cased = convert_reviews_to_lower_case(sampled_reviews_ratings_df, 'review_body')

In [23]:
reviews_lower_cased

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased
254374,4,The pens cam on time and I got the right numbe...,1.0,NaN,NaN,1.0,the pens cam on time and i got the right numbe...
1516628,4,Hey... It's a roll of STICKERS. What else can...,1.0,NaN,NaN,1.0,hey... it's a roll of stickers. what else can...
885182,4,It is as appears in the picture. It accommodat...,1.0,NaN,NaN,1.0,it is as appears in the picture. it accommodat...
2270302,4,I really loved this laser pointer. It is extr...,1.0,NaN,NaN,1.0,i really loved this laser pointer. it is extr...
1278832,4,In use with my Amazon Basics 3-Button corded U...,1.0,NaN,NaN,1.0,in use with my amazon basics 3-button corded u...
...,...,...,...,...,...,...,...
1810597,2,"First of all, let me say I am extremely happy ...",NaN,NaN,3.0,3.0,"first of all, let me say i am extremely happy ..."
2349422,2,This headset is amazing but the mic that you f...,NaN,NaN,3.0,3.0,this headset is amazing but the mic that you f...
235316,2,Smaller than expected. Didn't end up using the...,NaN,NaN,3.0,3.0,smaller than expected. didn't end up using the...
603714,2,it's nice that it collapses to about the size ...,NaN,NaN,3.0,3.0,it's nice that it collapses to about the size ...


In [24]:
print("reviews_lower_cased:")
generate_sample_reviews(reviews_lower_cased, 'lower_cased', 3)

reviews_lower_cased:
{'star_rating': 4, 'lower_cased': 'the pens cam on time and i got the right number and the colors pictured. however, the face on one of the pens was smudged when i was open. the ink flowed nicely as well.'}
{'star_rating': 4, 'lower_cased': "hey... it's a roll of stickers.  what else can i say?  they stick to the box, and conform to shipping regulations for small quantities of hazardous materials."}
{'star_rating': 4, 'lower_cased': "it is as appears in the picture. it accommodates my highlighters, erasure, grading pencil, and rule w/ ample space. the one quality/construction drawback is that a piece of cardboard feels like it is what is sewn b/n the layers of fabric on each side to give it shape. i'm concerned if it gets wet or bent/crashed under things in my bag, whether that will cause a structural problem."}


## Remove HTML and URLs

In [25]:
def remove_html_and_urls(df: pd.DataFrame, col_name: str):
    """Remove HTML and URLs from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the html_and_urls removed
    """
    
    # url_pattern = re.compile(r'https?://\S+|www\. \S+')

    cleaned_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
            # Check and remove HTML tags
            has_html = bool(re.search('<.*?>', text_review))
            if has_html == True:
                # print("Review", text_reviews_idx, "has HTML -- ", text_review)
                pass

            no_html_review = re.sub('<.*?>', ' ', text_review)
            # print("Review", text_reviews_idx, "without HTML -- ", no_html_review)
        
            # Check and remove URLs
            has_url = bool(re.search(r'http\S+', no_html_review))
            if has_url == True:
                # print("Review", text_reviews_idx, "has URL --", no_html_review)
                pass

            no_html_url_review = re.sub(r'http\S+', '', no_html_review)
            # print("Review", text_reviews_idx, "without HTML, URL -- ", no_html_url_review)
            # print()
            cleaned_reviews.append(no_html_url_review)
        else:
            # print(text_reviews_idx, text_review)
            cleaned_reviews.append(text_review)
            

    updated_df['without_html_urls'] = cleaned_reviews
    return updated_df

In [26]:
no_html_urls_df = remove_html_and_urls(reviews_lower_cased, 'lower_cased')

In [27]:
no_html_urls_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls
254374,4,The pens cam on time and I got the right numbe...,1.0,NaN,NaN,1.0,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...
1516628,4,Hey... It's a roll of STICKERS. What else can...,1.0,NaN,NaN,1.0,hey... it's a roll of stickers. what else can...,hey... it's a roll of stickers. what else can...
885182,4,It is as appears in the picture. It accommodat...,1.0,NaN,NaN,1.0,it is as appears in the picture. it accommodat...,it is as appears in the picture. it accommodat...
2270302,4,I really loved this laser pointer. It is extr...,1.0,NaN,NaN,1.0,i really loved this laser pointer. it is extr...,i really loved this laser pointer. it is extr...
1278832,4,In use with my Amazon Basics 3-Button corded U...,1.0,NaN,NaN,1.0,in use with my amazon basics 3-button corded u...,in use with my amazon basics 3-button corded u...
...,...,...,...,...,...,...,...,...
1810597,2,"First of all, let me say I am extremely happy ...",NaN,NaN,3.0,3.0,"first of all, let me say i am extremely happy ...","first of all, let me say i am extremely happy ..."
2349422,2,This headset is amazing but the mic that you f...,NaN,NaN,3.0,3.0,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...
235316,2,Smaller than expected. Didn't end up using the...,NaN,NaN,3.0,3.0,smaller than expected. didn't end up using the...,smaller than expected. didn't end up using the...
603714,2,it's nice that it collapses to about the size ...,NaN,NaN,3.0,3.0,it's nice that it collapses to about the size ...,it's nice that it collapses to about the size ...


In [28]:
print("without_html_urls:")
generate_sample_reviews(no_html_urls_df, 'without_html_urls', 3)

without_html_urls:
{'star_rating': 4, 'without_html_urls': 'the pens cam on time and i got the right number and the colors pictured. however, the face on one of the pens was smudged when i was open. the ink flowed nicely as well.'}
{'star_rating': 4, 'without_html_urls': "hey... it's a roll of stickers.  what else can i say?  they stick to the box, and conform to shipping regulations for small quantities of hazardous materials."}
{'star_rating': 4, 'without_html_urls': "it is as appears in the picture. it accommodates my highlighters, erasure, grading pencil, and rule w/ ample space. the one quality/construction drawback is that a piece of cardboard feels like it is what is sewn b/n the layers of fabric on each side to give it shape. i'm concerned if it gets wet or bent/crashed under things in my bag, whether that will cause a structural problem."}


## Remove Contractions

In [29]:
store_contractions = {
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he's": "he is",
    "isn't": "is not",
    "it's": "it is",
    "let's": "let us",
    "mustn't": "must not",
    "shan't": "shall not",
    "she's": "she is",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they're": "they are",
    "wasn't": "was not",
    "we're": "we are",
    "weren't": "were not",
    "won't": "will not",
    "wouldn't": "would not",
    "you're": "you are",
    "you'll": "you will",
    "you'd": "you would",
    "we'll": "we will",
    "we've": "we have",
    "we'd": "we would",
    "I'm": "I am",
    "i've": "I have",
    "I've": "I have",
    "I'd": "I would",
    "it'll": "it will",
    "they'll": "they will",
    "they've": "they have",
    "they'd": "they would",
    "he'll": "he will",
    "he'd": "he would",
    "she'll": "she will",
    "we'd": "we would",
    "we'll": "we will",
    "you've": "you have",
    "you'd": "you would",
    "you'll": "you will",
    "I'll": "I will",
    "I'd": "I would",
    "it's": "it is",
    "it'd": "it would",
    "i'm": "I am",
    "he's": "he is",
    "he'll": "he will",
    "she's": "she is",
    "she'll": "she will",
    "we're": "we are",
    "we've": "we have",
    "we'll": "we will",
    "you're": "you are",
    "you've": "you have",
    "you'll": "you will",
    "they're": "they are",
    "they've": "they have",
    "they'll": "they will",
    "that's": "that is",
    "that'll": "that will",
    "that'd": "that would",
    "who's": "who is",
    "who'll": "who will",
    "who'd": "who would",
    "what's": "what is",
    "what'll": "what will",
    "what'd": "what would",
    "when's": "when is",
    "when'll": "when will",
    "when'd": "when would",
    "where's": "where is",
    "where'll": "where will",
    "where'd": "where would",
    "why's": "why is",
    "why'll": "why will",
    "why'd": "why would",
    "how's": "how is",
    "how'll": "how will",
    "how'd": "how would"
}


In [30]:
def locate_and_replace_contractions(review):
    """Find the contractions to replace from a specific review

    Parameters
    ----------
    review: `str`
        A specific review

    Return
    ------
    non_contraction_review: `str`
        The updated specific review with contractions expanded
    
    """
    if isinstance(review, str):
        get_words = review.split()

        store_non_contraction_words = []

        for word in get_words:
            if word in store_contractions:
                non_contraction_form = store_contractions[word]
                # print(word, "-->", non_contraction_form)

                store_non_contraction_words.append(non_contraction_form)

            else:
                # print(word)
                store_non_contraction_words.append(word)

        non_contraction_review = ' '.join(store_non_contraction_words)
        return non_contraction_review
    else:
        return review


In [31]:
def remove_contractions(df:pd.DataFrame, col_name: str):
    """Remove contractions from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    without_contractions_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        # print("Review", text_reviews_idx, "with possible contraction(s) -- ", text_review)

        without_contraction = locate_and_replace_contractions(text_review)

        # print("Review", text_reviews_idx, "without contraction -- ", without_contraction)
        # print()

        without_contractions_reviews.append(without_contraction)

    updated_df['without_contractions'] = without_contractions_reviews
    return updated_df

In [32]:
no_contractions_df = remove_contractions(no_html_urls_df, 'without_html_urls')

In [33]:
no_contractions_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls,without_contractions
254374,4,The pens cam on time and I got the right numbe...,1.0,NaN,NaN,1.0,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...
1516628,4,Hey... It's a roll of STICKERS. What else can...,1.0,NaN,NaN,1.0,hey... it's a roll of stickers. what else can...,hey... it's a roll of stickers. what else can...,hey... it is a roll of stickers. what else can...
885182,4,It is as appears in the picture. It accommodat...,1.0,NaN,NaN,1.0,it is as appears in the picture. it accommodat...,it is as appears in the picture. it accommodat...,it is as appears in the picture. it accommodat...
2270302,4,I really loved this laser pointer. It is extr...,1.0,NaN,NaN,1.0,i really loved this laser pointer. it is extr...,i really loved this laser pointer. it is extr...,i really loved this laser pointer. it is extre...
1278832,4,In use with my Amazon Basics 3-Button corded U...,1.0,NaN,NaN,1.0,in use with my amazon basics 3-button corded u...,in use with my amazon basics 3-button corded u...,in use with my amazon basics 3-button corded u...
...,...,...,...,...,...,...,...,...,...
1810597,2,"First of all, let me say I am extremely happy ...",NaN,NaN,3.0,3.0,"first of all, let me say i am extremely happy ...","first of all, let me say i am extremely happy ...","first of all, let me say i am extremely happy ..."
2349422,2,This headset is amazing but the mic that you f...,NaN,NaN,3.0,3.0,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...
235316,2,Smaller than expected. Didn't end up using the...,NaN,NaN,3.0,3.0,smaller than expected. didn't end up using the...,smaller than expected. didn't end up using the...,smaller than expected. did not end up using th...
603714,2,it's nice that it collapses to about the size ...,NaN,NaN,3.0,3.0,it's nice that it collapses to about the size ...,it's nice that it collapses to about the size ...,it is nice that it collapses to about the size...


In [34]:
print("without_contractions:")
generate_sample_reviews(no_contractions_df, 'without_contractions', 3)

without_contractions:
{'star_rating': 4, 'without_contractions': 'the pens cam on time and i got the right number and the colors pictured. however, the face on one of the pens was smudged when i was open. the ink flowed nicely as well.'}
{'star_rating': 4, 'without_contractions': 'hey... it is a roll of stickers. what else can i say? they stick to the box, and conform to shipping regulations for small quantities of hazardous materials.'}
{'star_rating': 4, 'without_contractions': 'it is as appears in the picture. it accommodates my highlighters, erasure, grading pencil, and rule w/ ample space. the one quality/construction drawback is that a piece of cardboard feels like it is what is sewn b/n the layers of fabric on each side to give it shape. I am concerned if it gets wet or bent/crashed under things in my bag, whether that will cause a structural problem.'}


## Remove Non-alphabetical characters

In [35]:
def remove_non_alphabetical_characters(df:pd.DataFrame, col_name: str):
    """Remove Non-alphabetical characters from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the non-alphabetical characters removed
    """

    alphabetical_char_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        
        if isinstance(text_review, str):

            # Check for non-alphabetical characters
            has_non_alphabetical_char = bool(re.search(r'[^a-zA-Z]', text_review))
            if has_non_alphabetical_char == True:
                # print("Review", text_reviews_idx, "has HTML -- ", text_review)
                pass
            
            # Remove non-alphabetical characters
            with_alphabetical_char = re.sub(r'[^a-zA-Z\s]', ' ', text_review)
            # print("Review", text_reviews_idx, "has HTML -- ", with_alphabetical_char)
            alphabetical_char_reviews.append(with_alphabetical_char)
        else:
            alphabetical_char_reviews.append(text_review)

    updated_df['with_alpha_chars_only'] = alphabetical_char_reviews
    return updated_df

In [36]:
only_alpha_chars_df = remove_non_alphabetical_characters(no_contractions_df, 'without_contractions')

In [37]:
only_alpha_chars_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only
254374,4,The pens cam on time and I got the right numbe...,1.0,NaN,NaN,1.0,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...
1516628,4,Hey... It's a roll of STICKERS. What else can...,1.0,NaN,NaN,1.0,hey... it's a roll of stickers. what else can...,hey... it's a roll of stickers. what else can...,hey... it is a roll of stickers. what else can...,hey it is a roll of stickers what else can...
885182,4,It is as appears in the picture. It accommodat...,1.0,NaN,NaN,1.0,it is as appears in the picture. it accommodat...,it is as appears in the picture. it accommodat...,it is as appears in the picture. it accommodat...,it is as appears in the picture it accommodat...
2270302,4,I really loved this laser pointer. It is extr...,1.0,NaN,NaN,1.0,i really loved this laser pointer. it is extr...,i really loved this laser pointer. it is extr...,i really loved this laser pointer. it is extre...,i really loved this laser pointer it is extre...
1278832,4,In use with my Amazon Basics 3-Button corded U...,1.0,NaN,NaN,1.0,in use with my amazon basics 3-button corded u...,in use with my amazon basics 3-button corded u...,in use with my amazon basics 3-button corded u...,in use with my amazon basics button corded u...
...,...,...,...,...,...,...,...,...,...,...
1810597,2,"First of all, let me say I am extremely happy ...",NaN,NaN,3.0,3.0,"first of all, let me say i am extremely happy ...","first of all, let me say i am extremely happy ...","first of all, let me say i am extremely happy ...",first of all let me say i am extremely happy ...
2349422,2,This headset is amazing but the mic that you f...,NaN,NaN,3.0,3.0,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...
235316,2,Smaller than expected. Didn't end up using the...,NaN,NaN,3.0,3.0,smaller than expected. didn't end up using the...,smaller than expected. didn't end up using the...,smaller than expected. did not end up using th...,smaller than expected did not end up using th...
603714,2,it's nice that it collapses to about the size ...,NaN,NaN,3.0,3.0,it's nice that it collapses to about the size ...,it's nice that it collapses to about the size ...,it is nice that it collapses to about the size...,it is nice that it collapses to about the size...


In [38]:
print("with_alpha_chars_only:")
generate_sample_reviews(only_alpha_chars_df, 'with_alpha_chars_only', 3)

with_alpha_chars_only:
{'star_rating': 4, 'with_alpha_chars_only': 'the pens cam on time and i got the right number and the colors pictured  however  the face on one of the pens was smudged when i was open  the ink flowed nicely as well '}
{'star_rating': 4, 'with_alpha_chars_only': 'hey    it is a roll of stickers  what else can i say  they stick to the box  and conform to shipping regulations for small quantities of hazardous materials '}
{'star_rating': 4, 'with_alpha_chars_only': 'it is as appears in the picture  it accommodates my highlighters  erasure  grading pencil  and rule w  ample space  the one quality construction drawback is that a piece of cardboard feels like it is what is sewn b n the layers of fabric on each side to give it shape  I am concerned if it gets wet or bent crashed under things in my bag  whether that will cause a structural problem '}


## Remove extra spaces

In [39]:
def remove_extra_spaces(df:pd.DataFrame, col_name: str):
    """Remove extra spaces from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    single_spaced_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values
    # print(text_reviews)

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
        # Check if there are any extra spaces
            has_extra_space = bool(re.search(r' +', text_review))
            if has_extra_space == True:
                # print("Review", text_reviews_idx, "has extra space -- ", text_review)
                pass
            
            # Remove extra spaces
            single_spaced_review = re.sub(r' +', ' ', text_review)
            # print("Review", text_reviews_idx, "without extra space -- ", single_spaced_review)
            # print()
            
            single_spaced_reviews.append(single_spaced_review)
        else:
            single_spaced_reviews.append(text_review)

    updated_df['without_extra_space'] = single_spaced_reviews
    return updated_df

In [40]:
no_extra_space_df = remove_extra_spaces(only_alpha_chars_df, 'with_alpha_chars_only')

In [41]:
no_extra_space_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only,without_extra_space
254374,4,The pens cam on time and I got the right numbe...,1.0,NaN,NaN,1.0,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...
1516628,4,Hey... It's a roll of STICKERS. What else can...,1.0,NaN,NaN,1.0,hey... it's a roll of stickers. what else can...,hey... it's a roll of stickers. what else can...,hey... it is a roll of stickers. what else can...,hey it is a roll of stickers what else can...,hey it is a roll of stickers what else can i s...
885182,4,It is as appears in the picture. It accommodat...,1.0,NaN,NaN,1.0,it is as appears in the picture. it accommodat...,it is as appears in the picture. it accommodat...,it is as appears in the picture. it accommodat...,it is as appears in the picture it accommodat...,it is as appears in the picture it accommodate...
2270302,4,I really loved this laser pointer. It is extr...,1.0,NaN,NaN,1.0,i really loved this laser pointer. it is extr...,i really loved this laser pointer. it is extr...,i really loved this laser pointer. it is extre...,i really loved this laser pointer it is extre...,i really loved this laser pointer it is extrem...
1278832,4,In use with my Amazon Basics 3-Button corded U...,1.0,NaN,NaN,1.0,in use with my amazon basics 3-button corded u...,in use with my amazon basics 3-button corded u...,in use with my amazon basics 3-button corded u...,in use with my amazon basics button corded u...,in use with my amazon basics button corded usb...
...,...,...,...,...,...,...,...,...,...,...,...
1810597,2,"First of all, let me say I am extremely happy ...",NaN,NaN,3.0,3.0,"first of all, let me say i am extremely happy ...","first of all, let me say i am extremely happy ...","first of all, let me say i am extremely happy ...",first of all let me say i am extremely happy ...,first of all let me say i am extremely happy w...
2349422,2,This headset is amazing but the mic that you f...,NaN,NaN,3.0,3.0,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...
235316,2,Smaller than expected. Didn't end up using the...,NaN,NaN,3.0,3.0,smaller than expected. didn't end up using the...,smaller than expected. didn't end up using the...,smaller than expected. did not end up using th...,smaller than expected did not end up using th...,smaller than expected did not end up using the...
603714,2,it's nice that it collapses to about the size ...,NaN,NaN,3.0,3.0,it's nice that it collapses to about the size ...,it's nice that it collapses to about the size ...,it is nice that it collapses to about the size...,it is nice that it collapses to about the size...,it is nice that it collapses to about the size...


In [42]:
print("without_extra_space:")
generate_sample_reviews(no_extra_space_df, 'without_extra_space', 3)

without_extra_space:
{'star_rating': 4, 'without_extra_space': 'the pens cam on time and i got the right number and the colors pictured however the face on one of the pens was smudged when i was open the ink flowed nicely as well '}
{'star_rating': 4, 'without_extra_space': 'hey it is a roll of stickers what else can i say they stick to the box and conform to shipping regulations for small quantities of hazardous materials '}
{'star_rating': 4, 'without_extra_space': 'it is as appears in the picture it accommodates my highlighters erasure grading pencil and rule w ample space the one quality construction drawback is that a piece of cardboard feels like it is what is sewn b n the layers of fabric on each side to give it shape I am concerned if it gets wet or bent crashed under things in my bag whether that will cause a structural problem '}


# Pre-processing

## remove the stop words 

In [43]:
def filter_stop_words(df:pd.DataFrame, col_name: str):
    """Filter stop words out from all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    without_stop_words_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    stop_words = set(stopwords.words("english"))

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]

        if isinstance(text_review, str):
            text_review_words = word_tokenize(text_review) 

        

            # print("Before stop word removal", text_reviews_idx, " -- ", text_review)

            filtered_review = []

            for text_review_words_idx in range(len(text_review_words)):
                text_review_word = text_review_words[text_review_words_idx]
                
                # Check if review word is a stop word
                if text_review_word in stop_words:
                    # print("  Stop word -- ", text_review_word)
                    pass
                else:
                    # print(text_review_word, " -- is NOT a stop word in review")
                    filtered_review.append(text_review_word)

            
            filtered_review = " ".join(filtered_review)
            # print("After stop word removal", text_reviews_idx, " -- ", filtered_review)
            # print()
            
            without_stop_words_reviews.append(filtered_review)
        else:
            without_stop_words_reviews.append(text_review)
        

    updated_df['without_stop_words'] = without_stop_words_reviews
    return updated_df

In [44]:
no_stop_words_df = filter_stop_words(no_extra_space_df, 'without_extra_space')

In [45]:
no_stop_words_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only,without_extra_space,without_stop_words
254374,4,The pens cam on time and I got the right numbe...,1.0,NaN,NaN,1.0,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,pens cam time got right number colors pictured...
1516628,4,Hey... It's a roll of STICKERS. What else can...,1.0,NaN,NaN,1.0,hey... it's a roll of stickers. what else can...,hey... it's a roll of stickers. what else can...,hey... it is a roll of stickers. what else can...,hey it is a roll of stickers what else can...,hey it is a roll of stickers what else can i s...,hey roll stickers else say stick box conform s...
885182,4,It is as appears in the picture. It accommodat...,1.0,NaN,NaN,1.0,it is as appears in the picture. it accommodat...,it is as appears in the picture. it accommodat...,it is as appears in the picture. it accommodat...,it is as appears in the picture it accommodat...,it is as appears in the picture it accommodate...,appears picture accommodates highlighters eras...
2270302,4,I really loved this laser pointer. It is extr...,1.0,NaN,NaN,1.0,i really loved this laser pointer. it is extr...,i really loved this laser pointer. it is extr...,i really loved this laser pointer. it is extre...,i really loved this laser pointer it is extre...,i really loved this laser pointer it is extrem...,really loved laser pointer extremely bright go...
1278832,4,In use with my Amazon Basics 3-Button corded U...,1.0,NaN,NaN,1.0,in use with my amazon basics 3-button corded u...,in use with my amazon basics 3-button corded u...,in use with my amazon basics 3-button corded u...,in use with my amazon basics button corded u...,in use with my amazon basics button corded usb...,use amazon basics button corded usb optical mo...
...,...,...,...,...,...,...,...,...,...,...,...,...
1810597,2,"First of all, let me say I am extremely happy ...",NaN,NaN,3.0,3.0,"first of all, let me say i am extremely happy ...","first of all, let me say i am extremely happy ...","first of all, let me say i am extremely happy ...",first of all let me say i am extremely happy ...,first of all let me say i am extremely happy w...,first let say extremely happy seller response ...
2349422,2,This headset is amazing but the mic that you f...,NaN,NaN,3.0,3.0,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,headset amazing mic flip silver black advertis...
235316,2,Smaller than expected. Didn't end up using the...,NaN,NaN,3.0,3.0,smaller than expected. didn't end up using the...,smaller than expected. didn't end up using the...,smaller than expected. did not end up using th...,smaller than expected did not end up using th...,smaller than expected did not end up using the...,smaller expected end using
603714,2,it's nice that it collapses to about the size ...,NaN,NaN,3.0,3.0,it's nice that it collapses to about the size ...,it's nice that it collapses to about the size ...,it is nice that it collapses to about the size...,it is nice that it collapses to about the size...,it is nice that it collapses to about the size...,nice collapses size laptop used laptop stand l...


In [46]:
print("without_stop_words:")
generate_sample_reviews(no_stop_words_df, 'without_stop_words', 3)

without_stop_words:
{'star_rating': 4, 'without_stop_words': 'pens cam time got right number colors pictured however face one pens smudged open ink flowed nicely well'}
{'star_rating': 4, 'without_stop_words': 'hey roll stickers else say stick box conform shipping regulations small quantities hazardous materials'}
{'star_rating': 4, 'without_stop_words': 'appears picture accommodates highlighters erasure grading pencil rule w ample space one quality construction drawback piece cardboard feels like sewn b n layers fabric side give shape I concerned gets wet bent crashed things bag whether cause structural problem'}


## perform lemmatization  

- "A sentence with many words"
    - "words" -> word

In [47]:
def lemmentize_review(df:pd.DataFrame, col_name: str):
    """Lemmentize all reviews

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    Return
    ------
    df: `pd.DataFrame`
        An updated DataFrame with the extra spaces removed
    """
    
    lemmed_reviews = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    lem = WordNetLemmatizer()

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]   
        if isinstance(text_review, str):     
            words_in_review = word_tokenize(text_review) 

            # print("Before lem update", text_reviews_idx, " -- ", text_review)
            # print("Lemmed words", words_in_review)
            

            lemmed_sentence = []

            # Split review into words
            for lemmed_words_idx in range(len(words_in_review)):
                word = words_in_review[lemmed_words_idx]
                
                apply_lemmatization = lem.lemmatize(word)
                # print(apply_lemmatization)
                
                lemmed_sentence.append(apply_lemmatization)
                filtered_review = " ".join(lemmed_sentence)
        
            # print("After lem update -- ", filtered_review)
            # print()

            lemmed_reviews.append(filtered_review)
        else:
            lemmed_reviews.append(text_review)

    updated_df['lemmed_reviews'] = lemmed_reviews
    return updated_df

In [48]:
lemmed_df = lemmentize_review(no_stop_words_df, 'without_stop_words')

In [49]:
lemmed_df

,star_rating,review_body,negative_review_class,neutral_review_class,positive_review_class,review_class,lower_cased,without_html_urls,without_contractions,with_alpha_chars_only,without_extra_space,without_stop_words,lemmed_reviews
254374,4,The pens cam on time and I got the right numbe...,1.0,NaN,NaN,1.0,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,the pens cam on time and i got the right numbe...,pens cam time got right number colors pictured...,pen cam time got right number color pictured h...
1516628,4,Hey... It's a roll of STICKERS. What else can...,1.0,NaN,NaN,1.0,hey... it's a roll of stickers. what else can...,hey... it's a roll of stickers. what else can...,hey... it is a roll of stickers. what else can...,hey it is a roll of stickers what else can...,hey it is a roll of stickers what else can i s...,hey roll stickers else say stick box conform s...,hey roll sticker else say stick box conform sh...
885182,4,It is as appears in the picture. It accommodat...,1.0,NaN,NaN,1.0,it is as appears in the picture. it accommodat...,it is as appears in the picture. it accommodat...,it is as appears in the picture. it accommodat...,it is as appears in the picture it accommodat...,it is as appears in the picture it accommodate...,appears picture accommodates highlighters eras...,appears picture accommodates highlighter erasu...
2270302,4,I really loved this laser pointer. It is extr...,1.0,NaN,NaN,1.0,i really loved this laser pointer. it is extr...,i really loved this laser pointer. it is extr...,i really loved this laser pointer. it is extre...,i really loved this laser pointer it is extre...,i really loved this laser pointer it is extrem...,really loved laser pointer extremely bright go...,really loved laser pointer extremely bright go...
1278832,4,In use with my Amazon Basics 3-Button corded U...,1.0,NaN,NaN,1.0,in use with my amazon basics 3-button corded u...,in use with my amazon basics 3-button corded u...,in use with my amazon basics 3-button corded u...,in use with my amazon basics button corded u...,in use with my amazon basics button corded usb...,use amazon basics button corded usb optical mo...,use amazon basic button corded usb optical mou...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1810597,2,"First of all, let me say I am extremely happy ...",NaN,NaN,3.0,3.0,"first of all, let me say i am extremely happy ...","first of all, let me say i am extremely happy ...","first of all, let me say i am extremely happy ...",first of all let me say i am extremely happy ...,first of all let me say i am extremely happy w...,first let say extremely happy seller response ...,first let say extremely happy seller response ...
2349422,2,This headset is amazing but the mic that you f...,NaN,NaN,3.0,3.0,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,this headset is amazing but the mic that you f...,headset amazing mic flip silver black advertis...,headset amazing mic flip silver black advertis...
235316,2,Smaller than expected. Didn't end up using the...,NaN,NaN,3.0,3.0,smaller than expected. didn't end up using the...,smaller than expected. didn't end up using the...,smaller than expected. did not end up using th...,smaller than expected did not end up using th...,smaller than expected did not end up using the...,smaller expected end using,smaller expected end using
603714,2,it's nice that it collapses to about the size ...,NaN,NaN,3.0,3.0,it's nice that it collapses to about the size ...,it's nice that it collapses to about the size ...,it is nice that it collapses to about the size...,it is nice that it collapses to about the size...,it is nice that it collapses to about the size...,nice collapses size laptop used laptop

In [50]:
print("without_unlemmed_words:")
generate_sample_reviews(lemmed_df, 'lemmed_reviews', 3)

without_unlemmed_words:
{'star_rating': 4, 'lemmed_reviews': 'pen cam time got right number color pictured however face one pen smudged open ink flowed nicely well'}
{'star_rating': 4, 'lemmed_reviews': 'hey roll sticker else say stick box conform shipping regulation small quantity hazardous material'}
{'star_rating': 4, 'lemmed_reviews': 'appears picture accommodates highlighter erasure grading pencil rule w ample space one quality construction drawback piece cardboard feel like sewn b n layer fabric side give shape I concerned get wet bent crashed thing bag whether cause structural problem'}


# Extract Word Embeddings

In [51]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

import gensim.downloader as api
word_two_vec_model = api.load('word2vec-google-news-300')

2024-02-02 21:35:09,513 : INFO : loading projection weights from /Users/brinkley97/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz
2024-02-02 21:35:41,802 : INFO : KeyedVectors lifecycle event {'msg': 'loaded (3000000, 300) matrix of type float32 from /Users/brinkley97/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', 'binary': True, 'encoding': 'utf8', 'datetime': '2024-02-02T21:35:41.802502', 'gensim': '4.3.2', 'python': '3.11.4 (main, Jul 25 2023, 17:07:07) [Clang 14.0.3 (clang-1403.0.22.14.1)]', 'platform': 'macOS-13.6.3-x86_64-i386-64bit', 'event': 'load_word2vec_format'}


In [63]:
# retrieve the vocabulary of a model
words_in_model = []
for index, word in enumerate(word_two_vec_model.index_to_key):
    words_in_model.append(word)
    # if index == 7:
    #     break
    # print(f"word #{index}/{len(word_two_vec_model.index_to_key)} is {word}")

In [65]:
# words_in_model

In [101]:
def extract_word_embeddings(sentence: str):
    """Extract word embeddings per sentence
    
    """
    sentence_vectorized = []
    updated_sentence = sentence.split(" ")
    print("Before -- ", sentence, "\n", updated_sentence)

    # Split review into words
    for word_idx in range(len(updated_sentence)):
        word = updated_sentence[word_idx]
        # print(word)

        if word in words_in_model:
            print(word)
            vector = word_two_vec_model[word]
            print("vectorized word")
            sentence_vectorized.append(vector)
        else:
            print(False, word)
        
    return sentence_vectorized


In [102]:
def word_embeddings(df: pd.DataFrame, col_name: str, words_in_model: list):
    """Extract word embeddings

    Parameters
    ----------
    df: `pd.DataFrame`
        The data
    
    col_name: `str`
        Column with reviews

    words_in_model: `list`
        Words in Word2Vec model

    Return
    ------
    
    """
    
    sentence_vectorized = []
    updated_df = df.copy()
    text_reviews = df[col_name].values

    for text_reviews_idx in range(len(text_reviews)):
        text_review = text_reviews[text_reviews_idx]
        extracted_for_sentence = extract_word_embeddings(text_review)        
        sentence_vectorized.append(extracted_for_sentence)
        
    return sentence_vectorized

In [104]:
word_embedding_features = word_embeddings(lemmed_df[:3], 'lemmed_reviews', words_in_model)


Before --  pen cam time got right number color pictured however face one pen smudged open ink flowed nicely well 
 ['pen', 'cam', 'time', 'got', 'right', 'number', 'color', 'pictured', 'however', 'face', 'one', 'pen', 'smudged', 'open', 'ink', 'flowed', 'nicely', 'well']
pen
vectorized word
cam
vectorized word
time
vectorized word
got
vectorized word
right
vectorized word
number
vectorized word
color
vectorized word
pictured
vectorized word
however
vectorized word
face
vectorized word
one
vectorized word
pen
vectorized word
smudged
vectorized word
open
vectorized word
ink
vectorized word
flowed
vectorized word
nicely
vectorized word
well
vectorized word
Before --  hey roll sticker else say stick box conform shipping regulation small quantity hazardous material 
 ['hey', 'roll', 'sticker', 'else', 'say', 'stick', 'box', 'conform', 'shipping', 'regulation', 'small', 'quantity', 'hazardous', 'material']
hey
vectorized word
roll
vectorized word
sticker
vectorized word
else
vectorized word


In [107]:
word_embedding_features[2]

[array([ 0.08984375,  0.01031494, -0.22460938,  0.26367188, -0.08056641,
         0.0201416 ,  0.20605469, -0.14257812,  0.12695312, -0.03417969,
        -0.32617188, -0.0480957 , -0.03369141,  0.05224609, -0.08496094,
         0.14550781,  0.02868652,  0.24023438,  0.07275391, -0.16699219,
         0.05712891,  0.15039062, -0.08251953, -0.08935547, -0.07568359,
        -0.27929688, -0.02185059,  0.11669922, -0.00405884,  0.26367188,
         0.08349609, -0.06054688, -0.10302734,  0.27929688,  0.09765625,
         0.15332031,  0.02099609,  0.13378906,  0.00141144, -0.12695312,
         0.05053711, -0.15234375,  0.05932617, -0.09912109,  0.19042969,
         0.13964844, -0.00909424,  0.10205078,  0.19824219, -0.203125  ,
         0.04833984, -0.14746094, -0.01049805, -0.22363281,  0.11035156,
         0.15722656,  0.09863281, -0.11865234, -0.00860596,  0.07470703,
        -0.09033203,  0.25585938, -0.12597656, -0.15527344, -0.17089844,
        -0.109375  ,  0.07177734,  0.20703125, -0.1

## Split Features and Sentiment Labels

In [ ]:
sentiments = lemmed_df['sentiment']
sentiments.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(tf_idf_features, sentiments, test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

# Models + Evaluation Metrics

In [ ]:
def eval_accuracy(y_true, y_prediction):
    return sklearn.metrics.accuracy_score(y_true, y_prediction)

def eval_precision(y_true, y_prediction):
    return sklearn.metrics.precision_score(y_true, y_prediction)

def eval_recall(y_true, y_prediction):
    return sklearn.metrics.recall_score(y_true, y_prediction)

def eval_f1_score(y_true, y_prediction):
    return sklearn.metrics.f1_score(y_true, y_prediction)

In [ ]:
def train_eval_metric(y_train_true, y_train_predictions):
    accuracy = eval_accuracy(y_train_true, y_train_predictions)
    precision = eval_precision(y_train_true, y_train_predictions)
    recall = eval_recall(y_train_true, y_train_predictions)
    f1 = eval_f1_score(y_train_true, y_train_predictions)

    metrics_dict = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

    return metrics_dict

def test_eval_metric(y_test_true, y_test_predictions):
    accuracy = eval_accuracy(y_test_true, y_test_predictions)
    precision = eval_precision(y_test_true, y_test_predictions)
    recall = eval_recall(y_test_true, y_test_predictions)
    f1 = eval_f1_score(y_test_true, y_test_predictions)

    metrics_dict = {
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1
    }

    return metrics_dict

# Perceptron

In [ ]:
def perceptron_model(X_train, X_test, y_train, y_test): 

    technique = Perceptron(tol=1e-3, random_state=0)
    technique.fit(X_train, y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)


    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics


In [ ]:
perceptron_train_metrics, perceptron_test_metrics = perceptron_model(X_train, X_test, y_train, y_test)

In [ ]:
perceptron_train_metrics, perceptron_test_metrics

# SVM

In [ ]:
def svm_model(X_train, X_test, y_train, y_test): 

    technique = LinearSVC(tol=1e-3, random_state=0)
    technique.fit(X_train, y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)


    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics


In [ ]:
svm_train_metrics, svm_test_metrics = svm_model(X_train, X_test, y_train, y_test)

In [ ]:
svm_train_metrics, svm_test_metrics

# Logistic Regression

In [ ]:
def logistic_regression_model(X_train, X_test, y_train, y_test): 

    technique = LogisticRegression(random_state=0)
    technique.fit(X_train, y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)


    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics

In [ ]:
logistic_regression_train_metrics, logistic_regression_test_metrics = logistic_regression_model(X_train, X_test, y_train, y_test)

In [ ]:
logistic_regression_train_metrics, logistic_regression_test_metrics

# Naive Bayes

In [ ]:
def naive_bayes_model(X_train, X_test, y_train, y_test): 

    technique = MultinomialNB()
    technique.fit(X_train.toarray(), y_train)
    y_train_predictions = technique.predict(X_train)
    y_test_predictions = technique.predict(X_test)

    train_metrics = train_eval_metric(y_train, y_train_predictions)
    test_metrics = test_eval_metric(y_test, y_test_predictions)

    return train_metrics, test_metrics


In [ ]:
naive_bayes_train_metrics, naive_bayes_test_metrics = naive_bayes_model(X_train, X_test, y_train, y_test)

In [ ]:
naive_bayes_train_metrics, naive_bayes_test_metrics